In [78]:
from rdflib import URIRef, Graph
import pandas as pd

In [79]:
places = {
    "place": ["Q31", "Q145", "Q183", "Q31"],
    "start": ["1979-09-01", "1986-03-01", "2016-08-10", "2018-10-19"]
  }

In [80]:
p_df = pd.DataFrame(places)

In [81]:
p_df["start"] = pd.to_datetime(p_df["start"])
p_df

,place,start
0,Q31,1979-09-01
1,Q145,1986-03-01
2,Q183,2016-08-10
3,Q31,2018-10-19


In [82]:
values = ""
for p in places["place"]:
  values += 'wd:' + p + " "
print(values)

wd:Q31 wd:Q145 wd:Q183 wd:Q31 


In [83]:
q = """
    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>
    PREFIX wikibase: <http://wikiba.se/ontology#>
    PREFIX p: <http://www.wikidata.org/prop/>
    PREFIX ps: <http://www.wikidata.org/prop/statement/>
    PREFIX pq: <http://www.wikidata.org/prop/qualifier/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX bd: <http://www.bigdata.com/rdf#>

    SELECT DISTINCT ?country ?countryLabel ?lifeExpectancy ?area
    WHERE 
    { 
      SERVICE <https://query.wikidata.org/sparql> { 
        VALUES ?country { """ + values + """ } . 
        ?country wdt:P31 wd:Q3624078 . 
        ?country rdfs:label ?countryLabel . 
        OPTIONAL { ?country wdt:P2250 ?lifeExpectancy } 
        OPTIONAL { ?country wdt:P2046 ?area } 
      } 
      FILTER( lang(?countryLabel) = "en" ) 
    } 
"""

In [84]:
print(q)


    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>
    PREFIX wikibase: <http://wikiba.se/ontology#>
    PREFIX p: <http://www.wikidata.org/prop/>
    PREFIX ps: <http://www.wikidata.org/prop/statement/>
    PREFIX pq: <http://www.wikidata.org/prop/qualifier/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX bd: <http://www.bigdata.com/rdf#>

    SELECT DISTINCT ?country ?countryLabel ?lifeExpectancy ?area
    WHERE 
    { 
      SERVICE <https://query.wikidata.org/sparql> { 
        VALUES ?country { wd:Q31 wd:Q145 wd:Q183 wd:Q31 } . 
        ?country wdt:P31 wd:Q3624078 . 
        ?country rdfs:label ?countryLabel . 
        OPTIONAL { ?country wdt:P2250 ?lifeExpectancy } 
        OPTIONAL { ?country wdt:P2046 ?area } 
      } 
      FILTER( lang(?countryLabel) = "en" ) 
    } 



In [85]:
g = Graph()

In [86]:
qres = g.query(q)

In [87]:
print('Entries:', len(qres))

Entries: 3


In [88]:
df = pd.DataFrame(qres)

In [89]:
df.columns = ["IRI", "Country", "Life Expectancy", "Area"]

In [90]:
df[["Entity Rest", "Entity"]] = df["IRI"].str.rsplit("/", n=1, expand=True)

In [91]:
df

,IRI,Country,Life Expectancy,Area,Entity Rest,Entity
0,http://www.wikidata.org/entity/Q31,Belgium,80.99268,30528,http://www.wikidata.org/entity,Q31
1,http://www.wikidata.org/entity/Q183,Germany,80.8,357400,http://www.wikidata.org/entity,Q183
2,http://www.wikidata.org/entity/Q145,United Kingdom,80.9561,242495.406794,http://www.wikidata.org/entity,Q145


In [92]:
out = pd.merge(p_df,df, left_on="place", right_on="Entity").sort_values("start")

In [93]:
out["duration"] = out["start"].shift(-1) - out["start"]

In [94]:
out

,place,start,IRI,Country,Life Expectancy,Area,Entity Rest,Entity,duration
0,Q31,1979-09-01,http://www.wikidata.org/entity/Q31,Belgium,80.99268,30528,http://www.wikidata.org/entity,Q31,2373 days
2,Q145,1986-03-01,http://www.wikidata.org/entity/Q145,United Kingdom,80.9561,242495.406794,http://www.wikidata.org/entity,Q145,11120 days
3,Q183,2016-08-10,http://www.wikidata.org/entity/Q183,Germany,80.8,357400,http://www.wikidata.org/entity,Q183,800 days
1,Q31,2018-10-19,http://www.wikidata.org/entity/Q31,Belgium,80.99268,30528,http://www.wikidata.org/entity,Q31,NaT
